<a href="https://colab.research.google.com/github/CAllans-Git/Air-Quality-IoT-Project/blob/main/AhmedAbad_AQI_and_CO_LSTM_Model_for_Dashboard_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Data Cleaning and Preparation

In [ ]:

data = pd.read_csv('/content/drive/MyDrive/IoT Project- Air Quality/city_day_2020_cleaned.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
missing_percentage = (data.isnull().sum() / len(data)) * 100
missing_percentage_sorted = missing_percentage.sort_values(ascending=False)
columns_to_drop = missing_percentage_sorted[missing_percentage_sorted > 50].index
data.drop(columns=columns_to_drop, inplace=True)

numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
for column in numeric_columns:
    if data[column].isnull().sum() > 0:
        data[column].fillna(data[column].median(), inplace=True)

if 'AQI_Bucket' in data.columns:
    data['AQI_Bucket'].fillna(data['AQI_Bucket'].mode()[0], inplace=True)
if 'NO' in numeric_columns:
    scaler = MinMaxScaler()
    data['NO'] = scaler.fit_transform(data[['NO']])
X = data.drop(['AQI', 'AQI_Bucket', 'Date'], axis=1)
y = data['AQI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

file_path = '/content/drive/MyDrive/IoT Project- Air Quality/city_day_2020_cleaned.csv'
df = pd.read_csv(file_path)
ahmedabad_data = df[df['City'] == 'Ahmedabad']
print(ahmedabad_data.head())

        City        Date  PM2.5    PM10        NO    NO2    NOx   NH3    CO  \
0  Ahmedabad  2020-01-01  79.57  131.62  0.009523  12.64   8.99  13.5  3.78   
1  Ahmedabad  2020-01-02  72.59  128.86  0.009139  14.38   9.73  13.5  3.63   
2  Ahmedabad  2020-01-03  69.54  128.99  0.017920  15.13  12.65  13.5  7.06   
3  Ahmedabad  2020-01-04  66.20  138.47  0.022810  20.79  16.84  13.5  8.97   
4  Ahmedabad  2020-01-05  55.99  110.41  0.013696  15.34  11.53  13.5  5.41   

     SO2     O3  Benzene  Toluene    AQI AQI_Bucket  Year  Month  Day  
0  27.70  23.67     4.21    31.42  216.0       Poor  2020      1    1  
1  23.96  23.67     3.71    31.14  162.0   Moderate  2020      1    2  
2  35.78  23.66     4.78    31.14  220.0       Poor  2020      1    3  
3  38.98  23.65     4.12    31.14  254.0       Poor  2020      1    4  
4  45.83  23.61     3.30    31.14  255.0       Poor  2020      1    5  


In [ ]:

X = ahmedabad_data[['Year', 'Month', 'Day']]
y = ahmedabad_data['AQI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 13184.87158108108


#Modeling
## Each model is evaluated separaely and a summary table of all model performances is provided at the end of this section

# Model 2: LSTM for Time Series Forecasting


In [ ]:

aqi_values = data['AQI'].fillna(method='ffill').values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_aqi = scaler.fit_transform(aqi_values)

In [ ]:

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
n_input = 14
batch_size = 1
generator = TimeseriesGenerator(scaled_aqi, scaled_aqi, length=n_input, batch_size=batch_size)

In [ ]:

scaled_aqi = np.array(scaled_aqi)
np.random.seed(42)
sample_indices = np.random.choice(len(scaled_aqi), 1000, replace=False)
sampled_scaled_aqi = scaled_aqi[sample_indices]
generato
model2 = Sequential([
    LSTM(50, activation='relu', input_shape=(n_input, 1)),
    Dropout(0.2),
    Dense(1)
])
model2.compile(optimizer='adam', loss='mean_squared_error')
model2.fit(generator, epochs=10, steps_per_epoch=len(generator), verbose=0)
scores_2 = model2.evaluate(generator, verbose=0)
print("Model 2 - LSTM model")
print(f"Loss: {scores_2}")


Model 2 - LSTM model
Loss: 0.0035286915954202414


In [ ]:

import math
n_input = 14
batch_size = 1
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_input, 1)),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
train_generator = TimeseriesGenerator(sampled_scaled_aqi, sampled_scaled_aqi, length=n_input, batch_size=batch_size)
model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator), verbose=0)
predictions = np.array([model.predict(train_generator[i][0], verbose=0) for i in range(len(train_generator))])
actual_values = np.array([train_generator[i][1] for i in range(len(train_generator))])
predictions = predictions.reshape(predictions.shape[0], predictions.shape[2])
actual_values = actual_values.reshape(actual_values.shape[0], actual_values.shape[2])
mse = mean_squared_error(actual_values, predictions)
rmse = math.sqrt(mse)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

MSE: 0.0037009313660254306
RMSE: 0.06083528060283301


In [ ]:

file_path = '/content/drive/MyDrive/IoT Project- Air Quality/city_day_2020_cleaned.csv'
data = pd.read_csv(file_path)
ahmedabad_data = data[data['City'] == 'Ahmedabad'].dropna(subset=['AQI'])
ahmedabad_data['Date'] = pd.to_datetime(ahmedabad_data['Date'])
aqi_values = ahmedabad_data['AQI'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_aqi_values = scaler.fit_transform(aqi_values)
n_input = 14
batch_size = 1
generator = TimeseriesGenerator(scaled_aqi_values, scaled_aqi_values, length=n_input, batch_size=batch_size)
model = Sequential([
    LSTM(50, input_shape=(n_input, 1)),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(generator, epochs=10, steps_per_epoch=len(generator), verbose=0)
predictions = model.predict(generator, verbose=0).flatten()
predicted_aqi = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
prediction_dates = ahmedabad_data['Date'][n_input:].reset_index(drop=True)
predicted_aqi_df = pd.DataFrame({
    'Date': prediction_dates,
    'Actual AQI': ahmedabad_data['AQI'][n_input:].reset_index(drop=True),
    'Predicted AQI': predicted_aqi
})
predicted_aqi_df.to_csv('/content/drive/MyDrive/ahmedabad_aqi_predictions_lstm.csv', index=False)
mse = mean_squared_error(predicted_aqi_df['Actual AQI'], predicted_aqi_df['Predicted AQI'])
rmse = math.sqrt(mse)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(predicted_aqi_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MSE: 27919.215460731484
RMSE: 167.09044096156873
        Date  Actual AQI  Predicted AQI
0 2020-01-15       142.0     220.715515
1 2020-01-16       152.0     200.601929
2 2020-01-17       334.0     192.389465
3 2020-01-18       547.0     231.535172
4 2020-01-19       375.0     304.774414


In [ ]:

predicted_aqi_df.to_csv('/content/drive/MyDrive/IoT Project- Air Quality/ahmedabad_aqi_predictions_lstm.csv', index=False)
print("DataFrame saved to Google Drive at '/content/drive/MyDrive/IoT Project- Air Quality/ahmedabad_aqi_predictions_lstm.csv'.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame saved to Google Drive at '/content/drive/MyDrive/IoT Project- Air Quality/ahmedabad_aqi_predictions_lstm.csv'.


In [ ]:

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/IoT Project- Air Quality/city_day_2020_cleaned.csv'
data = pd.read_csv(file_path)

ahmedabad_data = data[data['City'] == 'Ahmedabad'].dropna(subset=['CO'])

co_values = ahmedabad_data['CO'].values.reshape(-1, 1)
ahmedabad_data = data[data['City'] == 'Ahmedabad'].dropna(subset=['CO'])
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_co_values = scaler.fit_transform(co_values)
n_input = 14
batch_size = 1
generator = TimeseriesGenerator(scaled_co_values, scaled_co_values, length=n_input, batch_size=batch_size)
model = Sequential([
    LSTM(50, input_shape=(n_input, 1)),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(generator, epochs=10, steps_per_epoch=len(generator), verbose=0)
predictions = model.predict(generator, verbose=0).flatten()
predicted_co = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
predicted_co_df = pd.DataFrame({
    'Date': ahmedabad_data['Date'][n_input:].reset_index(drop=True),
    'Actual CO': ahmedabad_data['CO'][n_input:].reset_index(drop=True),
    'Predicted CO': predicted_co
})
predicted_co_df.to_csv('/content/drive/MyDrive/ahmedabad_co_predictions_lstm.csv', index=False)
predicted_co_df.to_csv('/content/drive/MyDrive/IoT Project- Air Quality/ahmedabad_co_predictions_lstm.csv', index=False)

mse = mean_squared_error(predicted_co_df['Actual CO'], predicted_co_df['Predicted CO'])
rmse = math.sqrt(mse)
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(predicted_co_df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MSE: 161.8917264883205
RMSE: 12.723667965186788
        Date  Actual CO  Predicted CO
0 2020-01-15       3.59      6.466534
1 2020-01-16       3.75      6.144241
2 2020-01-17      20.06      5.984624
3 2020-01-18      24.98      9.616166
4 2020-01-19      13.05     12.836783
